In [1]:
import numpy as np
import pandas as pd
from numba import jit
from itertools import compress
import os

In [4]:
def startend(filename, header):
    if header:
        start = pd.read_csv(filename, sep='\t', skiprows=1, engine='c', usecols=[1], names=['start'], squeeze=True, dtype={'start':'int'}).to_numpy()
        end = pd.read_csv(filename, sep='\t', skiprows=1, engine='c', usecols=[2], names=['end'], squeeze=True, dtype={'end':'int'}).to_numpy()
    else:
        start = pd.read_csv(filename, sep='\t', engine='c', usecols=[1], names=['start'], squeeze=True, dtype={'start':'int'}).to_numpy()
        end = pd.read_csv(filename, sep='\t', engine='c', usecols=[2], names=['end'], squeeze=True, dtype={'end':'int'}).to_numpy()
    
    subtraction = np.subtract(end, start)  
    df = pd.DataFrame({'start':start, 'subtraction':subtraction})
    uq_id = pd.Series(df.start.astype(str) + df.subtraction.astype(str)).astype(np.int64).to_numpy()
    
    return uq_id

In [5]:
def refalt(filename, header):
    if header:
        ref_alt = pd.read_csv(filename, sep='\t', skiprows=1, engine='c', usecols=[3,4], names=['ref', 'alt'], dtype={'ref':'string','alt':'string'})
    else:
        ref_alt = pd.read_csv(filename, sep='\t', engine='c', usecols=[3,4], names=['ref', 'alt'], dtype={'ref':'string','alt':'string'})
    
    ref_alt['ref'] = ref_alt['ref'].apply(lambda x: sum([ord(i) for i in x]))
    ref_alt['alt'] = ref_alt['alt'].apply(lambda x: sum([ord(i) for i in x]))
    
    uq_id = pd.Series(ref_alt.ref.astype(str) + ref_alt.alt.astype(str)).astype(int).to_numpy()
    return uq_id

In [49]:
# start end ref alt = sera
def sera(filename, header):
    
    if header:
        start = pd.read_csv(filename, sep='\t', skiprows=1, engine='c', usecols=[1], names=['start'], squeeze=True, dtype={'start':'int'}).to_numpy()
        end = pd.read_csv(filename, sep='\t', skiprows=1, engine='c', usecols=[2], names=['end'], squeeze=True, dtype={'end':'int'}).to_numpy()
        ref_alt = pd.read_csv(filename, sep='\t', skiprows=1, engine='c', usecols=[3,4], names=['ref', 'alt'], dtype={'ref':'string','alt':'string'})
    else:
        start = pd.read_csv(filename, sep='\t', engine='c', usecols=[1], names=['start'], squeeze=True, dtype={'start':'int'}).to_numpy()
        end = pd.read_csv(filename, sep='\t', engine='c', usecols=[2], names=['end'], squeeze=True, dtype={'end':'int'}).to_numpy()
        ref_alt = pd.read_csv(filename, sep='\t', engine='c', usecols=[3,4], names=['ref', 'alt'], dtype={'ref':'string','alt':'string'})
    
    subtraction = np.subtract(end, start)  
    df = pd.DataFrame({'start':start, 'subtraction':subtraction})
    uq_id = pd.Series(df.start.astype(str) + df.subtraction.astype(str)).astype(np.int64).to_numpy()
    del df
    
    ref_alt['ref'] = ref_alt['ref'].apply(lambda x: sum([ord(i) for i in x]))
    ref_alt['alt'] = ref_alt['alt'].apply(lambda x: sum([ord(i) for i in x]))
    
    uq_id2 = pd.Series(ref_alt.ref.astype(str) + ref_alt.alt.astype(str)).astype(np.longlong).to_numpy()
    
    df = pd.DataFrame({'startend':uq_id, 'refalt':uq_id2})
    uq_id3 = pd.Series(df.startend.astype(str) + df.refalt.astype(str)).astype(np.longlong).to_numpy()
    
    return uq_id3

In [46]:
def subtraction(first, second):
    return np.in1d(first, second)

In [13]:
# test anno is 22249/ pci variant, 2 is 22246/ pci parent
uq_startend = startend('test_anno.txt', True)
uq_startend2 = startend('test_anno2.txt', True)
print(uq_startend)
print(uq_startend2)

[    654330     659990     695690 ... 2489181120 2489181290 2489182500]
[    660190    4507500    4511090 ... 2489177310 2489183000 2489184960]


In [14]:
uq_refalt = refalt('test_anno.txt', True)
uq_refalt2 = refalt('test_anno2.txt', True)
print(uq_refalt)
print(uq_refalt2)

[7184 7184 8465 ... 6765 6765 6765]
[6571 6784 7165 ... 6765 6765 6784]


In [39]:
startend_sub = subtraction(uq_startend, uq_startend2)
print(np.size(startend_sub))
print(np.count_nonzero(startend_sub)) # true count

157598
33258


In [40]:
refalt_sub = subtraction(uq_refalt, uq_refalt2)
print(np.size(refalt_sub))
print(np.count_nonzero(refalt_sub))

157598
156699


In [38]:
print(refalt_sub[:10])
print(startend_sub[:10])
print(valid := np.logical_and(refalt_sub[:10], startend_sub[:10]))
print(np.size(valid) - np.count_nonzero(valid))

[ True  True  True  True  True  True  True  True  True  True]
[False False False False False False  True  True  True  True]
[False False False False False False  True  True  True  True]
6


In [42]:
# intersect boolean arrays... doesnt work bc all refalt nonunique
intersect_sub = np.logical_and(refalt_sub, startend_sub)
print(np.count_nonzero(intersect_sub))

33239


In [50]:
corr_uq = sera('test_anno.txt', True)
corr_uq2 = sera('test_anno2.txt', True)
print(corr_uq)
print(corr_uq2)

OverflowError: int too big to convert